In [42]:
import gzip
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [8]:
class ResonanceDataset(Dataset):

    def __init__(self, images, targets):

        # store cross-sectional value as a function of (E, theta)
        self.images = images # shape: (num_samples, 1, num_theta, num_E)

        # targets stores a set of resonance data for each sample
        # consists of: energy, width (Gamma), dominant l
        self.targets = targets # shape: (num_samples, 3)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        
        image = self.images[idx]
        target = {key: val[idx] for key, val in self.targets.items()}

        return image, target

In [38]:
def load_images(train_path):
    
    with open(train_path, 'r') as f:
        data = json.load(f)
    
    n = len(data)
    
    for i in range(n):
    
        levels = data[i]['levels'][0]
        observable_sets = data[i]['observable_sets']
    
        if i != 0:
            continue
    
        pp_in_index = observable_sets[0]['pp_in_index']
        pp_out_index = observable_sets[0]['pp_out_index']
        points = observable_sets[0]['points'][0]